Installing the dependencies

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
import numpy as np
import pandas as pd
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
from joblib import load
import os

model_path = '/content/solar_svr_model.joblib'
assert os.path.exists(model_path), "Model file missing!"

model_data = load(model_path)
weather_model = model_data['model']
scaler_X = model_data['scaler_X']  # Critical for preprocessing
scaler_y = model_data['scaler_y']  # Critical for postprocessing

In [ ]:
from pydantic import BaseModel
from fastapi import FastAPI, HTTPException
import pickle  # Added missing import
import json

app = FastAPI()

# Your input model
class WeatherInput(BaseModel):
    temp: float
    humidity: float
    dew: float
    precip: float
    cloudcover: float
    solarradiation: float
    solarenergy: float


@app.post('/weather_prediction')
async def predict_solar_energy(input_parameters: WeatherInput):
    # Check if model and scalers are loaded
    if weather_model is None or scaler_X is None or scaler_y is None:
        raise HTTPException(
            status_code=500,
            detail="Model or scalers not loaded properly"
        )

    try:
        # Convert input to dictionary (Pydantic V2 compatible)
        input_dict = input_parameters.model_dump()

        # Create input array in correct feature order
        input_values = np.array([[
            input_dict['temp'],
            input_dict['humidity'],
            input_dict['dew'],
            input_dict['precip'],
            input_dict['cloudcover'],
            input_dict['solarradiation'],
            input_dict['solarenergy']
        ]])

        # Scale the input features
        input_scaled = scaler_X.transform(input_values)

        # Get prediction (still scaled)
        pred_scaled = weather_model.predict(input_scaled)

        # Inverse transform to get actual KWH value
        pred_kwh = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1))

        # Return properly formatted response
        return {
            "prediction": float(pred_scaled[0]),  # Scaled value (0-1)
            "prediction_kwh": float(pred_kwh[0][0]),  # Actual KWH value
            "message": "Success",
            "status_code": 200
        }

    except KeyError as e:
        raise HTTPException(
            status_code=400,
            detail=f"Missing required feature: {str(e)}"
        )
    except Exception as e:
        raise HTTPException(
            status_code=400,
            detail=f"Prediction error: {str(e)}"
        )

In [ ]:
ngrok.set_auth_token("YOUR_API_(OR)_AUTH_KEY")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

In [ ]:
# import pickle
# import os # Import os to check if the file exists

# # Assuming you have the trained model object ready
# # For example, if it's a scikit-learn model:
# # from sklearn.svm import SVR
# # svr_model = SVR() # Replace with your actual trained model

# # Check if the model object exists in your environment.
# # If not, you need to retrain or load the model correctly before saving.
# # Example of how to create a dummy model for demonstration:
# from sklearn.svm import SVR
# from sklearn.datasets import make_regression
# X, y = make_regression(n_samples=100, n_features=10, random_state=0)
# # Create and train a simple SVR model for demonstration if you don't have yours
# try:
#     # Try to access the existing svr_model variable
#     svr_model_to_save = svr_model
#     print("Using the existing 'svr_model' variable to save.")
# except NameError:
#     # If 'svr_model' variable does not exist, create a dummy one
#     print("The variable 'svr_model' does not exist. Creating a dummy SVR model and saving it.")
#     svr_model_to_save = SVR()
#     svr_model_to_save.fit(X, y)


# # --- Re-saving the model using pickle ---
# try:
#     with open('svr_model.pkl', 'wb') as f:
#         pickle.dump(svr_model_to_save, f)
#     print("Model successfully re-saved as 'svr_model.pkl' using pickle.")
# except Exception as e:
#     print(f"Error saving model with pickle: {e}")

# # --- Loading the model using pickle ---
# try:
#     # loading the saved model
#     # Ensure the file exists before trying to load
#     if os.path.exists('svr_model.pkl'):
#         with open('svr_model.pkl', 'rb') as f:
#             diabetes_model = pickle.load(f)
#         print("Model successfully loaded from 'svr_model.pkl' using pickle.")
#     else:
#         print("Error: 'svr_model.pkl' not found after attempting to save.")
# except UnpicklingError as e:
#     print(f"UnpicklingError occurred during load: {e}. The file might still be corrupted or not a valid pickle file.")
# except Exception as e:
#     print(f"An unexpected error occurred during load: {e}")

# # --- Alternative: Using joblib (often preferred for scikit-learn models) ---
# # First, install joblib if you haven't already
# # !pip install joblib

# # import joblib

# # # --- Re-saving the model using joblib ---
# # try:
# #     joblib.dump(svr_model_to_save, 'svr_model_joblib.pkl')
# #     print("Model successfully re-saved as 'svr_model_joblib.pkl' using joblib.")
# # except Exception as e:
# #     print(f"Error saving model with joblib: {e}")

# # # --- Loading the model using joblib ---
# # try:
# #     # Ensure the file exists before trying to load
# #     if os.path.exists('svr_model_joblib.pkl'):
# #         diabetes_model_joblib = joblib.load('svr_model_joblib.pkl')
# #         print("Model successfully loaded from 'svr_model_joblib.pkl' using joblib.")
# #     else:
# #          print("Error: 'svr_model_joblib.pkl' not found after attempting to save.")
# # except Exception as e:
# #     print(f"Error loading model with joblib: {e}")

In [ ]:
# import joblib
# # Load saved objects
# saved_data = joblib.load('svr_model.pkl')
# model = saved_data['model']
# scaler_X = saved_data['scaler_X']
# scaler_y = saved_data['scaler_y']

# def predict_kwh(temp, humidity, dew, precip, cloudcover, solarradiation, solarenergy):
#     # 1. Scale input
#     input_scaled = scaler_X.transform([[temp, humidity, dew, precip, cloudcover, solarradiation, solarenergy]])

#     # 2. Predict
#     pred_scaled = model.predict(input_scaled)

#     # 3. Unscale prediction
#     return scaler_y.inverse_transform(pred_scaled.reshape(-1, 1))[0][0]

# # Usage
# predict_kwh(25.5, 60.0, 18.0, 0.0, 30.0, 450.0, 1.8)  # Returns unscaled KWH prediction